## Read in a TonicNet Numpy file and perform it on a csound instrument
### Performed on Finger Piano and Woodwinds
This notebook includes code to experiment with arpeggios in the finger piano. My intent is to search for ways to reduce the reliance on 1/16th note rhythms and increase the variety of rhythmic features.

In [1]:
import numpy as np
import time
from importlib import reload
import matplotlib.pyplot as plt
import os
import muspy
from mido import MidiFile
import fluidsynth
from IPython.display import Audio, display
import sys
import logging
import music21 as m21
sys.path.insert(0, '/home/prent/Dropbox/Tutorials/Diamond_Music')
import diamond_music_utils as dmu
from numpy.random import default_rng
rng = np.random.default_rng()
from IPython.display import Audio, display
numpy_dir = os.path.join('eval', 'numpy_chorales')
midi_dir = os.path.join('eval', 'midi_samples')
print(f'{numpy_dir = }, {midi_dir = }')
keys = np.array(['C♮', 'D♭', 'D♮', 'E♭', 'E♮', 'F♮', 'G♭', 'G♮', 'A♭', 'A♮', 'B♭', 'B♮'])
soundfont = '../coconet-pytorch/font.sf2' 

numpy_dir = 'eval/numpy_chorales', midi_dir = 'eval/midi_samples'


In [2]:
def find_root_mode(file_name):
    # file_name = os.path.join(midi_dir,midi_name)
    # print(f'determining root & mode for: {file_name = }')
    s = m21.converter.parse(file_name)
    fis = str(s.analyze('key'))
    # print(f'music21 says: {fis = }')
    key_name, mode = fis.split()
    # print(f'after split. {key_name = }, {mode = }')
    i = 0
    root = 9999
    for key in keys:
        if key.upper() == key_name.upper():
            root = i
            # print(f'found a match between music21 {key_name = }, and {key = } as {root = }, {mode = }')
        i += 1
    if root == 9999: root = 8
    return (root, mode)

In [103]:
file_num = 4788  
audio_file = os.path.join('/home/prent/Music/sflib', 'wave.wav')
print(f'{audio_file = }')
midi_name = 'sample' + str(file_num) + '.mid'
path = os.path.join(midi_dir, midi_name)
root, mode = find_root_mode(path)
muspy.read_midi(path, backend='mido', duplicate_note_mode='fifo')
music = muspy.read_midi(path) # convert mido to muspy music
print(f'read {midi_name = } and loaded it into a muspy Music structure. {len(music.tracks) = }, {round(muspy.pitch_class_entropy(music),2) = }') # {muspy_structure.from_dict() = } 
muspy.write_audio(audio_file, music, 'wav', soundfont, 44100, 3) # use fluidsynth to create a wave file
audio_link = Audio(audio_file)
display(audio_link) # display the audio widget
print(f'{file_num = }, {root = }, {keys[root] = }, {mode = }')

audio_file = '/home/prent/Music/sflib/wave.wav'
read midi_name = 'sample4788.mid' and loaded it into a muspy Music structure. len(music.tracks) = 4, round(muspy.pitch_class_entropy(music),2) = 2.24


fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]


file_num = 4788, root = 8, keys[root] = 'A♭', mode = 'major'


In [4]:
# track the name, the start time, and the instrument number
voice_time = {
    "fp1": {"full_name": "finger piano 1", "start": 0, "csound_voice": 1, "time_tracker_number": 0},
    "fp2": {"full_name": "finger piano 2", "start": 0, "csound_voice": 1, "time_tracker_number": 1},
    "fp3": {"full_name": "finger piano 3", "start": 0, "csound_voice": 1, "time_tracker_number": 2},
    "fp4": {"full_name": "finger piano 4", "start": 0, "csound_voice": 1, "time_tracker_number": 3},
    "fp5": {"full_name": "finger piano 5", "start": 0, "csound_voice": 1, "time_tracker_number": 4},
    "fp6": {"full_name": "finger piano 6", "start": 0, "csound_voice": 1, "time_tracker_number": 5},
    "bn1": {"full_name": "bass finger piano 1", "start": 0, "csound_voice": 2, "time_tracker_number": 6},
    "bn2": {"full_name": "bass finger piano 2", "start": 0, "csound_voice": 2, "time_tracker_number": 7},
    "bd1": {"full_name": "bass balloon drum 1", "start": 0, "csound_voice": 3, "time_tracker_number": 8},
    "bd2": {"full_name": "bass balloon drum 2", "start": 0, "csound_voice": 3, "time_tracker_number": 9},
    "bd3": {"full_name": "bass balloon drum 3", "start": 0, "csound_voice": 3, "time_tracker_number": 10},
    "bd4": {"full_name": "bass balloon drum 4", "start": 0, "csound_voice": 3, "time_tracker_number": 11},
    "bm1": {"full_name": "medium balloon drum 1", "start": 0, "csound_voice": 4, "time_tracker_number": 12},
    "bm2": {"full_name": "medium balloon drum 2", "start": 0, "csound_voice": 4, "time_tracker_number": 13},
    "bm3": {"full_name": "medium balloon drum 3", "start": 0, "csound_voice": 4, "time_tracker_number": 14},
    "bm4": {"full_name": "medium balloon drum 4", "start": 0, "csound_voice": 4, "time_tracker_number": 15},
    "bh1": {"full_name": "high balloon drum 1", "start": 0, "csound_voice": 5, "time_tracker_number": 16},
    "bh2": {"full_name": "high balloon drum 2", "start": 0, "csound_voice": 5, "time_tracker_number": 17},
    "bh3": {"full_name": "high balloon drum 3", "start": 0, "csound_voice": 5, "time_tracker_number": 18},
    "bh4": {"full_name": "high balloon drum 4", "start": 0, "csound_voice": 5, "time_tracker_number": 19},
    "bf1": {"full_name": "bass flute 1", "start": 0, "csound_voice": 6, "time_tracker_number": 20},
    "bf2": {"full_name": "bass flute 2", "start": 0, "csound_voice": 6, "time_tracker_number": 21},
    "bf3": {"full_name": "bass flute 3", "start": 0, "csound_voice": 6, "time_tracker_number": 22},
    "bf4": {"full_name": "bass flute 4", "start": 0, "csound_voice": 6, "time_tracker_number": 23},
    "ob1": {"full_name": "oboe", "start": 0, "csound_voice": 7, "time_tracker_number": 24},
    "cl1": {"full_name": "clarinet", "start": 0, "csound_voice": 8, "time_tracker_number": 25},
    "bs1": {"full_name": "bassoon", "start": 0, "csound_voice": 9, "time_tracker_number": 26},
    "fr1": {"full_name": "french horn", "start": 0, "csound_voice": 10, "time_tracker_number": 27},
    "btg": {"full_name": "baritone guitar", "start": 0, "csound_voice": 11, "time_tracker_number": 28},
    "ob2": {"full_name": "oboe", "start": 0, "csound_voice": 7, "time_tracker_number": 29},
    "cl2": {"full_name": "clarinet", "start": 0, "csound_voice": 8, "time_tracker_number": 30},
    "bs2": {"full_name": "bassoon", "start": 0, "csound_voice": 9, "time_tracker_number": 31},
    "fr2": {"full_name": "french horn", "start": 0, "csound_voice": 10, "time_tracker_number": 32},   
}
CSD_FILE = 'ball4.csd'
tempo = 72
jupyter_log = 'ball4.log'
dmu.start_logger(jupyter_log)
CS_LOGNAME = 'csound_ball4.log'

In [104]:
file_num = 4788
# print(f'{chosen = }, {file_num = }')
print(f'midi source\tentropy\tlength\tkey {[values[[0, 8, 10, 11]] for values in metrics if values[0] == "sample" + str(file_num) + ".mid"]}')
numpy_file_name = os.path.join(numpy_dir, 'chorale' + str(file_num) + '.npy') # load the numpy array 
chorale = np.load(numpy_file_name)
print(f'{chorale.shape = }')

midi source	entropy	length	key [array(['sample4788.mid', '2.241', '7168', 'D major'], dtype='<U32')]
chorale.shape = (4, 28)


In [5]:
def midi_to_name(notes):
    # convert midi notes to scale degrees C to B as 0-12
    names = np.array([keys[note_12et % 12] for note_12et in notes])
    return (names)

In [17]:
def build_voices_notes_features(chorale):
      # this function is passed a numpy array of note numbers in midi format, four per time step SATB. input is of the form: voice, midi_note
      # it converts the midi numbers into two features: notes and octaves
      # It returns a numpy array of (voices, notes, octaves)
      octave = np.array([midi_number // 12 for midi_number in chorale])
      note = np.array([midi_number % 12 for midi_number in chorale])
      # return np.stack((note, octave), axis = 0) #  (2, 4, 73) feature, voice, note
      # return np.stack((note, octave), axis = 1) #  (4, 2, 73) voice, feature, note
      return np.stack((note, octave), axis = 2) # voice_note_feature.shape = (4, 73, 2)

In [7]:
# the goal of this function is to string a bunch of octave changes together 
# target_array_value = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
#                 [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,-1,-1,-1,-1,-1,-1,-1,-1],
#                 [-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2],
#                 [2,2,2,2,2,2,2,2,-1,-1,-1,-1,-1,-1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1]])
#
# once target_array_value is assembled, you will add it to the octave_array, then the result is multiplied by the octave_mask
# the octave_mask is the same dimension and contains only 0's and 1's
# dimension of target_array_value must be (voices, chorale.shape[1])
# repeats is how many times each 1/16th note is repeated in the chorale.
# notice that row 1 is the same as row 0 rolled by some number that is a multiple of repeats. 
# This could be used to create all voices, just roll the initial array to create additional dimensions.
# How long you stay in place must be a multiple of repeats.
# 
def build_octave_alteration_mask(repeats, voices, chorale, \
            octave_stretch = 4, p1 = [.2, .3, .3, .2], octave_reduce = 2, stay = 7, \
            p2 =  [.1, .1, .2, .2, .2, .1, .1]):
    octave_alteration_mask = np.empty(0, dtype = int)
    done = False
    while not done:
        some_octave_change = rng.choice(octave_stretch, p = p1) - octave_reduce # returns a single number 0,1,2,3,4,5 - 2 = -2,-1,0,1,2,3 rarely hitting the largest and smallest values
        some_repeat_value = (1 + rng.choice(stay, p = p2)) * repeats
        repeated_octave_change = np.repeat(some_octave_change, some_repeat_value, axis = 0)
        octave_alteration_mask = np.concatenate((octave_alteration_mask, repeated_octave_change), axis = 0)
        done = octave_alteration_mask.shape[0] > chorale.shape[1] 
    octave_alteration_mask = octave_alteration_mask[:chorale.shape[1]] # cut off the excess array elements
    return np.array([np.roll(octave_alteration_mask, iteration * repeats, axis = 0) for iteration in np.arange(voices)])

# the following is designed to build a mask that sets long strings of zeros and ones.
# the result is an octave mask that has long held notes followed by long rests, at least as long as the repeats value
#
def build_long_mask(repeats, voices, chorale, p1 = [.5, .5], stay = 7, p2 =  [.1, .1, .2, .2, .2, .1, .1]):
    octave_alteration_mask = np.empty(0, dtype = int)
    done = False
    while not done:
        some_octave_change = rng.choice(2, p = p1)  # returns a zero or one
        some_repeat_value = (1 + rng.choice(stay, p = p2)) * repeats # pick a number between 1 and 7 inclusive to stay a zero or one
        repeated_octave_mask = np.repeat(some_octave_change, some_repeat_value, axis = 0)
        octave_alteration_mask = np.concatenate((octave_alteration_mask, repeated_octave_mask), axis = 0)
        done = octave_alteration_mask.shape[0] > chorale.shape[1] 
    octave_alteration_mask = octave_alteration_mask[:chorale.shape[1]] # cut off the excess array elements
    return np.array([np.roll(octave_alteration_mask, iteration * repeats, axis = 0) for iteration in np.arange(voices)])

### What was the temperature of the files:
      1.0 for sample0 through sample4099
      1.2 for sample4100 through sample4499
      0.8 for sample4500 through sample4899 

In [218]:
def add_features(voices_notes_features, guev_array):
      gls, gls_p, ups, ups_p, env, env_p, vel, vel_p = np.moveaxis(guev_array, 0, 0)
      # print(f'gls, gls_p, ups, ups_p, env, env_p, vel, vel_p: {[value for value in (gls, gls_p, ups, ups_p, env, env_p, vel, vel_p)]}')
      # voices_notes_features shape = (4, 73, 2) (voices, notes, features)
      break_point = voices_notes_features.shape[1] // env.shape[0] # # of notes divided by the shape of env
      # set the indeciis for the features to zero
         
      # print(f'{break_point = }')
      notes = voices_notes_features[:,:,0] # the 0th feature is the note #
      # print(f'{notes.shape = }')
      octaves = voices_notes_features[:,:,1] # the 1th feature is the octave #
      # print(f'{octaves.shape = }')
      # set the features for each note in the chorale, all voices
      gliss = np.zeros(notes.shape, dtype = int)
      upsample = np.zeros(notes.shape, dtype = int)   
      envelope = np.zeros(notes.shape, dtype = int)
      velocity = np.zeros(notes.shape, dtype = int)     
      # for every voice
      for voice in np.arange(notes.shape[0]):
            # for every note in the voice
            # print(f'first note in voice: {voice = } {notes[voice, 0] = }')
            gls_i = 0
            ups_i = 0
            env_i = 0
            vel_i = 0   
            prev_note = notes[voice, 0]
            prev_gls = rng.choice(gls[gls_i], p = gls_p[gls_i])
            prev_ups = rng.choice(ups[ups_i], p = ups_p[ups_i])
            prev_env = rng.choice(env[env_i], p = env_p[env_i])
            prev_vel = rng.choice(vel[vel_i], p = vel_p[vel_i])
            for note in np.arange(notes.shape[1]):
                  if notes[voice, note] != prev_note:
                        # print(f'new note: {voice = }, {note = }, {notes[voice, note] = }, {gls_i = }')
                        gliss[voice, note] = rng.choice(gls[gls_i], p = gls_p[gls_i])
                        upsample[voice, note] = rng.choice(ups[ups_i], p = ups_p[ups_i])
                        envelope[voice, note] = rng.choice(env[env_i], p = env_p[env_i])
                        velocity[voice, note] = rng.choice(vel[vel_i], p = vel_p[vel_i])
                        prev_note = notes[voice, note]
                        prev_gls = gliss[voice, note]
                        prev_ups = upsample[voice, note]
                        prev_env = envelope[voice, note]
                        prev_vel = velocity[voice, note]
                  else:
                        # print(f'{notes[voice, note] = }')
                        gliss[voice, note] = prev_gls
                        upsample[voice, note] = prev_ups
                        envelope[voice, note] = prev_env
                        velocity[voice, note] = prev_vel

                  if note % break_point == 0 and note > 0:
                        print(f'{note = }, {note % break_point = }, {env_i = } {env[env_i]}')
                        gls_i = np.min((gls.shape[0] - 1, gls_i + 1))
                        ups_i = np.min((ups.shape[0] - 1, ups_i + 1))
                        env_i = np.min((env.shape[0] - 1, env_i + 1))
                        vel_i = np.min((vel.shape[0] - 1, vel_i + 1))
                        print(f'increased indeciis: {env_i = } {env[env_i]}')
                       
      return np.stack((notes, octaves, gliss, upsample, envelope, velocity), axis = 0) 




In [225]:
def finger_piano_part(chorale, repeats, voice_names, tpq, four_features):
      print(f'in finger_piano_part. {chorale.shape = }, {repeats = }, {voice_names = }')
      voices = voice_names.shape[0] # if you want it to last twice as long, pretend there are twice as many voices: voice_names.shape[0] * 2
      chorale = np.repeat(chorale, repeats, axis = 1) # make each note 4 times as long
      print(f'after repeating each note {repeats = }: {chorale.shape = }')
      chorale = np.repeat(chorale, voices // 4, axis = 0)
      print(f'after doubling voices: {chorale.shape = }')
      print(f'{chorale.shape = }')
      volume_array = np.full(chorale.shape[1], 7)
      # revised 3/22/23
      voices_notes_features = build_voices_notes_features(chorale)
      print(f'{voices_notes_features.shape = }') # all must be the same shape, (2,2), (3,3) etc. square only allowed
      gls = np.array([[0, 0],[0, 0],[0, 0]])
      gls_p = np.array([[.5, .5], [.5, .5], [.5, .5]])
      ups = np.array([[-1, 0],[0, 1],[1, 2]])
      ups_p = np.array([[.5, .5], [.5, .5], [.5, .5]])
      env = np.array([[1, 0], [2, 8], [16, 17]])
      env_p = np.array([[.5, .5], [.6, .4], [.5, .5]])
      vel = np.array([[71, 74], [74, 77], [75, 78]])
      vel_p = np.array([[.5, .5], [.5, .5], [.5, .5]])
      guev_array = np.stack((gls, gls_p, ups, ups_p, env, env_p, vel, vel_p), axis = 0)
      print(f'In finger piano. feature array after stack. {guev_array.shape = }') # guev_array.shape = (8, 3, 2)
      # determine values for gliss, upsample, envelope, and velocity arrays
      notes_features_6 = add_features(voices_notes_features, guev_array)
      print(f'feature values and counts in this order: notes, octaves, gliss, upsample, envelope, velocity')
      print(*[np.unique(feature, return_counts=True) for feature in notes_features_6], sep = '\n')
      octave_array = notes_features_6[1] # all the octaves for all the voices, notes
      # print(f'{dmu.largest_evenly_divisible(chorale.shape[1], 15) = }') # i forgot why I wanted to know this. Perhaps on the next chorale it will be needed.
      # create an array to mask some notes so that octave = 0, which makes them silent
      probs = [[.2, .8],[.3, .7],[.4,.6],[.01,.99]] # from mostly ones, to half and half Note: p = [.01, .99] makes a very sparce collection of held notes.
      density_function = np.array([rng.choice(2, size = (voices, (repeats // 2) - 1), p = prob) for prob in probs]).reshape(8,-1) # This will be used over and over again throughout the piece.
      print(f'after first creation: {density_function.shape = }')
      density_function = np.concatenate((np.ones((voices, 1), dtype = int), density_function), axis = 1) # (8,30)
      print(f'after adding ones to first beat: {density_function.shape = }')
      density_function = np.tile(density_function, chorale.shape[1] // density_function.shape[1] + 1)
      print(f'{octave_array.shape = }, {density_function.shape = }')
      octave_alteration_mask = build_octave_alteration_mask(repeats, voices, chorale)
      print(f'{octave_alteration_mask.shape = }')
      octave_array += octave_alteration_mask
      print(f'octaves after spread: ')
      print(*[np.unique(octave, return_counts=True) for octave in octave_array], sep = '\n')
      notes_features_6[1] = octave_array * density_function[:, :octave_array.shape[1]] # make the octave go to zero for some percent of the notes
      print(f'{notes_features_6.shape = }')
      notes_features_15 = dmu.piano_roll_to_notes_features(notes_features_6, volume_array, voice_names, tpq, voice_time)
      print(f'{notes_features_15.shape = }')
      return notes_features_15


In [230]:
def woodwinds_part(chorale, repeats, voice_names, tpq, four_features):
      print(f'in woodwinds_part. {chorale.shape = }, {repeats = }, {voice_names = }')
      voices = voice_names.shape[0] # if you want it to last twice as long, pretend there are twice as many voices: voice_names.shape[0] * 2
      chorale = np.repeat(chorale, repeats, axis = 1) # make each note 4 times as long
      print(f'after repeating each note {repeats = }: {chorale.shape = }')
      chorale = np.repeat(chorale, voices // 4, axis = 0)
      print(f'after doubling voices: {chorale.shape = }')
      # chorale = np.concatenate((chorale, np.zeros((4,1),dtype=int)),axis = 1) # add a bit at the end so you don't loose the ending
      print(f'{chorale.shape = }')
      volume_array = np.full(chorale.shape[1], 5)
      
      # revised 3/22/23
      voices_notes_features = build_voices_notes_features(chorale)
      print(f'{voices_notes_features.shape = }') # all must be the same shape, (2,2), (3,3) etc. square only allowed
      gls = np.array([[0, 0],[0, 0],[0, 0]])
      gls_p = np.array([[.5, .5], [.5, .5], [.5, .5]])
      ups = np.array([[-1, 0],[0, 1],[1, 2]])
      ups_p = np.array([[.5, .5], [.5, .5], [.5, .5]])
      env = np.array([[1, 16], [6, 9], [0, 5]])
      env_p = np.array([[.5, .5], [.5, .5], [.5, .5]])
      vel = np.array([[65, 68], [64, 69], [63, 70]])
      vel_p = np.array([[.5, .5], [.5, .5], [.5, .5]])
      guev_array = np.stack((gls, gls_p, ups, ups_p, env, env_p, vel, vel_p), axis = 0)
      print(f'In woodwinds. feature array after stack. {guev_array.shape = }') # guev_array.shape = (8, 3, 2)
      # determine values for gliss, upsample, envelope, and velocity arrays
      notes_features_6 = add_features(voices_notes_features, guev_array)
      print(f'feature values and counts in this order: notes, octaves, gliss, upsample, envelope, velocity')
      print(*[np.unique(feature, return_counts=True) for feature in notes_features_6], sep = '\n')
      
      print(f'{notes_features_6.shape = }') # notes_features_6.shape = (8, 915, 6) (voices, notes, features)
      octave_array = notes_features_6[1] # all the octaves for all the voices, notes

      print(f'prior to masking: {[np.average(voice) for voice in octave_array]}')
      octave_alteration_mask = build_long_mask(repeats, voices, chorale)
      print(f'{octave_array.shape = }, {octave_alteration_mask.shape = }') 
      # wrong: octave_array.shape = (915, 6), 
      # right: octave_alteration_mask.shape = (8, 915)
      octave_array *= octave_alteration_mask # multiply by an array of zeros and ones with the same shape as the octave_array
      print(f'after masking: {[np.average(voice) for voice in octave_array]}')
      print(f'{notes_features_6.shape = }')
      notes_features_15 = dmu.piano_roll_to_notes_features(notes_features_6, volume_array, voice_names, tpq, voice_time)
      print(f'{notes_features_15.shape = }')
      return notes_features_15


In [231]:
#
# Main note generating module
#
# transform the piano roll chorale into csound input
# load a chorale from the numpy arrays
# midi_nums = np.load('midi_nums.npy') # these are the chorales that have tracks that are 50,000 long
metrics = np.load('metrics.npy')
# print(f'{metrics.shape = }')
# chosen = 11
# file_num = midi_nums[chosen] # should be 4321, 4411
file_num = 4435 # b minor # 1844 # D major # a short one in D major: 4788
# print(f'{chosen = }, {file_num = }')
print(f'midi source\tentropy\tlength\tkey {[values[[0, 8, 10, 11]] for values in metrics if values[0] == "sample" + str(file_num) + ".mid"]}')
numpy_file_name = os.path.join(numpy_dir, 'chorale' + str(file_num) + '.npy') # load the numpy array 
chorale = np.load(numpy_file_name)
print(f'average midi number for each voice: {[np.average(voice) for voice in chorale] = }') # just for fun, sanity check
repeats = 15
chorale = np.concatenate((chorale, np.repeat(chorale[:,-1], repeats, axis = 0).reshape(4, repeats)), axis = 1) # add a bit at the end so you make sure you have a nice bunch of repeated chords at the end. Fade out later
print(f'{chorale.shape = }')
unique_note_names, count_of_note_names = np.unique(np.array([voice % 12 for voice in chorale]), return_counts=True)
print(f'{unique_note_names = }')
print(f'unique note names: {np.array([keys[note] for note in unique_note_names])}')
print(f'{count_of_note_names = }')
voice_note_feature = build_voices_notes_features(chorale)
print(f'{voice_note_feature.shape = }')

wood = True
fing = True
# initialize the instrument arrays
dmu.init_voice_start_times(voice_time) # start from the begining - set all instruments to start at time zero
stored_gliss = dmu.init_stored_gliss() # resets the global glissando array and the global current_gliss_table variable to 800
finger_pianos = np.array(['fp1', 'fp2', 'fp3', 'fp4', 'fp5', 'fp6', 'bn1', 'bn2'])
wood_winds = np.array(["ob1","cl1","fr1","bs1","ob2","cl2","fr2","bs2"])

notes_features_15 = np.empty((0,15), dtype = int) # start with an empty array you can concatenate onto.
if fing:
    notes_features_15 = np.concatenate((notes_features_15, finger_piano_part(chorale, repeats, finger_pianos, 0.25, four_features)), axis = 0)
if wood:
    notes_features_15 = np.concatenate((notes_features_15, woodwinds_part(chorale, repeats, wood_winds, 0.25, four_features)), axis = 0)
# now that you have the voices, assign note start times from durations of notes in a voice
notes_features_final, voice_time = dmu.fix_start_times(notes_features_15, voice_time)
print(f'{notes_features_final.shape = }')
# print out the duration of each voice by voice
print("finger_pianos: ", *[(inst, dmu.format_seconds_to_minutes(voice_time[inst]["start"])) for inst in finger_pianos if voice_time[inst]["start"] > 0],sep='\t')
print("wood_winds: ", *[(inst, dmu.format_seconds_to_minutes(voice_time[inst]["start"])) for inst in wood_winds if voice_time[inst]["start"] > 0],sep='\t')
notes_features = dmu.send_to_csound_file(notes_features_final, voice_time, CSD_FILE, tempos = 't0 ' + str(tempo), tempo = tempo, print_only = 2000) 
# plot the density of each note in the 12TET

# plt.hist(unique_note_names, weights=count_of_note_names, bins = 12)
# plt.show()

midi source	entropy	length	key [array(['sample4435.mid', '3.49', '106752', 'b minor'], dtype='<U32')]
average midi number for each voice: [np.average(voice) for voice in chorale] = [64.378896882494, 65.38129496402878, 59.89448441247002, 50.04796163069545]
chorale.shape = (4, 432)
unique_note_names = array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
unique note names: ['C♮' 'D♭' 'D♮' 'E♭' 'E♮' 'F♮' 'G♭' 'G♮' 'A♭' 'A♮' 'B♭' 'B♮']
count_of_note_names = array([139, 172, 205, 103, 185,  60, 159, 139, 161, 162,  68, 175])
voice_note_feature.shape = (4, 432, 2)
in finger_piano_part. chorale.shape = (4, 432), repeats = 15, voice_names = array(['fp1', 'fp2', 'fp3', 'fp4', 'fp5', 'fp6', 'bn1', 'bn2'],
      dtype='<U3')
after repeating each note repeats = 15: chorale.shape = (4, 6480)
after doubling voices: chorale.shape = (8, 6480)
chorale.shape = (8, 6480)
voices_notes_features.shape = (8, 6480, 2)
In finger piano. feature array after stack. guev_array.shape = (8, 3, 2)
note = 2160, note

In [232]:
csound = True
play = False
ship = False
if csound:
    logging.info(f'logging csound output to csound_{CS_LOGNAME}')
    result = os.system(f'csound new_output.csd -Ocsound_{CS_LOGNAME}') # give it a log file to write csound messages to.
    result = os.system(f"egrep 'invalid|replacing|range|error|cannot|rtevent|overall' csound_{CS_LOGNAME}") # inspect the log for important information
if play: result = os.system(f'play ~/Music/sflib/ball4.wav')
if ship: 
    os.system(f'scp ~/Music/sflib/ball4.wav prent@192.168.68.57:~/Music/sflib')
    os.system(f'scp ~/Music/sflib/ball4.eps prent@192.168.68.57:~/Music/sflib')

end of score.		   overall amps:  27937.7  25147.4
	   overall samples out of range:        0        0
0 errors in performance


In [233]:
version = 6
trim = True 
if trim:
    result = os.system(f'sh trim.sh ball4 {version}')

/home/prent/Music/sflib
total 3.9G
-rw-r--r--. 1 prent prent 4.1M Mar 23 14:41 ball4.eps
-rw-r--r--. 1 prent prent 341M Mar 23 14:41 ball4.wav
-rw-r--r--. 1 prent prent 461M Mar 23 10:13 ball4-t5.wav
-rw-r--r--. 1 prent prent 404M Mar 23 10:13 ball4a-c.wav
-rw-r--r--. 1 prent prent 461M Mar 22 15:45 ball4-t4.wav
-rw-r--r--. 1 prent prent 1.1M Mar 22 13:37 wave.wav
-rw-r--r--. 1 prent prent  78M Mar 21 15:52 ball4-t3.wav
-rw-r--r--. 1 prent prent 462M Mar 20 16:10 ball4-t2.wav
-rw-r--r--. 1 prent prent 462M Mar 19 16:05 ball4-t1.wav


export SFDIR="/home/prent/Music/sflib"
echo $SFDIR
ls $SFDIR -lth | head -n 10
csound -U sndinfo $SFDIR/"$1".wav
0dBFS level = 32768.0
--Csound version 6.16 (double samples) Oct 17 2022
[commit: none]
libsndfile-1.1.0
util sndinfo:
/home/prent/Music/sflib/ball4.wav:
	srate 44100, stereo, 24 bit WAV, 1350.401 seconds
	(59552675 sample frames)
end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 0.000s, CPU: 0.000s
csound "$1"c.csd 
0dBFS level = 32768.0
--Csound version 6.16 (double samples) Oct 17 2022
[commit: none]
libsndfile-1.1.0
UnifiedCSD:  ball4c.csd
STARTING FILE
Creating options
Creating orchestra
closing tag
Creating score
rtaudio: pulseaudio module enabled
rtmidi: ALSA Raw MIDI module enabled
sorting score ...
	... done
graphics suppressed, ascii substituted
0dBFS level = 32768.0
orch now loaded
audio buffered in 256 sample-frame blocks
writing 1536-byte blks of 24bit ints to /ho